# Prepare data

In [1]:
from script.func_split_data import split_data

In [2]:
split_df = split_data(data_path='Cosmenet_product_20231018', data_csv='datas_20231018.csv')
split_df.split_data()
split_df.report_train_test_split()
print()
split_df.report_train_test_val_split()

amount of all data : 60196
amount of all class : 4178
amount of data 2-8 img : 1548
amount of 2-8 img class : 278
amount of data more 8 img : 58631
amount of more 8 img class : 3883
amount of data & class only one : 17

amount of train split : 38474
amount of train split class : 3184
amount of test split : 9620
amount of test split class : 3184
amount of train val : 8430
amount of train val class : 699
amount of test val : 2107
amount of test val class : 699
amount of train val mix : 9204
amount of train val mix class : 977
amount of test val mix : 2881
amount of test val mix class : 977


In [3]:
df_train_split, df_test_split = split_df.get_train_test()
df_train_val_mix, df_test_val_mix = split_df.get_validate()

In [4]:
df_train_split.head(1)

,file_names,labels,images_path
0,11596_2.jpg,11596,/app/nfs_clientshare/Datasets/Cosmenet_product...


In [5]:
df = split_df.get_dict()

# extract and put to elasticsearch

In [12]:
import torch
from transformers import ViTImageProcessor, ViTModel

from script.func_extract_feature import select_transformers_model, pipeline_transformer
from script.tool import ROOT_NFS_TEST

2023-11-02 16:26:24.200631: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 16:26:24.849905: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 16:26:24.849981: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 16:26:24.854020: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-02 16:26:25.189031: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

In [20]:
device = torch.device('cpu')

model, preprocess = select_transformers_model(ViTModel, ViTImageProcessor, 
                                              pretrain=ROOT_NFS_TEST / 'weights/vit_gg_lr2e-05_eu_9ep_0_95099acc')
model.load_state_dict(torch.load('weights/temp_epoch/vitgg_lr2e05_ep5_loss0.0.pth')['model_state_dict'])
vit_gg_trained_lr2e_05_pipe = pipeline_transformer(layer="last_hidden_state", row=0, device=device)
vit_gg_trained_lr2e_05_pipe.selct_model(model, preprocess)
vit_gg_trained_lr2e_05_pipe.report_test()

runtime : 758.3239078521729 ms
outputs layers : odict_keys(['last_hidden_state', 'pooler_output'])
shape last_hidden_state : torch.Size([1, 197, 768])
shape pooler_output : torch.Size([1, 768])


In [ ]:
from script.func_query_score import extract_to_es

In [ ]:
ext_ep3_crop = extract_to_es('vitgg_lr2e05_ep5_loss0.0')
ext_ep3_crop.check_index_exist()
ext_ep3_crop.put_all_tag(vit_gg_trained_lr2e_05_pipe, df, replace=False, crop=False)

/tmp/ipykernel_343377/2355362069.py:39: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  err = self.es.indices.create(index=self.name_index, body=body_product)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'vitgg_lr2e05_ep5_loss0.0'}


  0%|          | 0/38474 [00:00<?, ?it/s]

/tmp/ipykernel_343377/2751088454.py:35: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  self.es.index(index=self.name_index, id=tag+"_"+str(n), body=data)


# Extract Elasticnet b1

In [39]:
from script.func_extract_feature import select_timm_model, pipeline_timm

In [40]:
model, preprocess = select_timm_model('efficientnet_b1', num_classes=0, pretrain=True)
eff_pipe = pipeline_timm(device=device)
eff_pipe.selct_model(model, preprocess)
eff_pipe.report_test()

runtime : 179.80599403381348 ms
Output shape at layer : torch.Size([1, 1280])


In [41]:
eff = extract_to_es('efficientnet_b1')
eff.check_index_exist(dims=1280)
eff.put_all_tag(eff_pipe, df, replace=False, crop=False)

index efficientnet_b1 already exists


  0%|          | 0/38474 [00:00<?, ?it/s]

put tag train_split success


  0%|          | 0/9620 [00:00<?, ?it/s]

put tag test_split success


  0%|          | 0/9204 [00:00<?, ?it/s]

/tmp/ipykernel_330647/2751088454.py:35: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  self.es.index(index=self.name_index, id=tag+"_"+str(n), body=data)


put tag train_val success


  0%|          | 0/2881 [00:00<?, ?it/s]

put tag test_val success


# measure_score

In [ ]:
from script.func_query_score import measure_score, report_image

In [ ]:
url = "192.168.1.97:9200"

In [ ]:
measure_15k = measure_score("efficientnet_b1", url=url)
result_15k = measure_15k.report_acc()

## Show image

In [ ]:
report = report_image(name_index="vitgg_lr2e05_ep3_loss0.0", url=url)
rand_id_split, rand_id_val = report.show_report(num_product=5, top_n=5, collapse=True)